In [4]:
### import and initialize modules ###
%load_ext autoreload
%autoreload 2

%matplotlib inline
# # if you want to use this, run  'conda install -c conda-forge scalene' for your conda env
# %load_ext scalene 
%load_ext line_profiler
import sys
sys.path.append("source/code")
from registration_project import rigid_reg_cc_demo, affine_reg_cc_demo, affine_reg_mi_demo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The scalene extension is already loaded. To reload it, use:
  %reload_ext scalene


In [ ]:
### import and read data ###
path = 'source\data\image_data'



In [ ]:
### Registration ###
# We can proba
%lprun -f affine_reg_cc_demo affine_reg_cc_demo(r"../data/image_data/1_1_t1.tif",r"../data/image_data/1_1_t1_d.tif",False) # example

In [ ]:
### Statistical analysis ###

# input: list of computation times per type of registration
# input: list of computation times per formula

# output: significance testing between the types of registration
# output: power analysis of significance testing
